In [1]:
# Imports

import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import tree, metrics
from sklearn import preprocessing
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Load data set, drop col with numbers, drop high collinearity columns
# Don't want to drop religion but unsure how to scale Nulls
data = pd.read_csv("marital_satisfaction_data_wrangled_final.csv")
data = data.drop(['country','Unnamed: 0','spouse_satisfaction','relationship_satisfaction','religion'], axis=1)
data.describe()

,sex,age,marriage_duration_years,num_children_total,num_children_inhome,edu_level,material_situation,religiosity,pension,enjoy_spouse_company,...,spouse_love,marital_satisfaction,natl_pride_in_parents,natl_pride_in_children,natl_aging_parents_live_with_children,natl_children_live_at_home_marraige,indv_pride_in_parents,indv_pride_in_children,indv_aging_parents_live_with_children,indv_children_live_at_home_marraige
count,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,...,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000,7178.000000
mean,1.533157,40.664948,14.753970,1.766648,1.466982,4.242965,0.385205,4.037197,0.343271,1.558233,...,1.659097,1.739203,1.507662,2.013514,0.452912,1.316801,1.807746,2.114517,0.601560,1.010588
std,0.498934,11.446839,11.594163,1.307748,1.383066,0.949635,0.831985,1.777629,1.951630,0.803552,...,0.748116,1.452407,1.460362,1.430655,1.819215,1.766169,1.529274,1.470525,1.959335,1.922027
min,1.000000,17.000000,0.000000,0.000000,0.000000,1.000000,-2.000000,1.000000,-3.000000,-2.000000,...,-2.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000
25%,1.000000,32.000000,5.000000,1.000000,0.000000,4.000000,0.000000,3.000000,-1.000000,1.000000,...,2.000000,1.000000,1.000000,2.000000,-1.000000,1.000000,1.000000,2.000000,-1.000000,0.000000
50%,2.000000,39.000000,12.000000,2.000000,1.000000,5.000000,0.000000,4.000000,1.000000,2.000000,...,2.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000
75%,2.000000,49.000000,23.000000,2.000000,2.000000,5.000000,1.000000,5.000000,2.000000,2.000000,...,2.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000,3.000000
max,2.000000,88.000000,70.000000,12.000000,13.000000,5.000000,2.000000,7.000000,3.000000,2.000000,...,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000


In [3]:
X = data.drop(['marital_satisfaction'], axis=1)
y = data.marital_satisfaction

In [4]:
cols= list(X.columns)
scaler = preprocessing.RobustScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=cols)
X

,sex,age,marriage_duration_years,num_children_total,num_children_inhome,edu_level,material_situation,religiosity,pension,enjoy_spouse_company,...,spouse_romance,spouse_love,natl_pride_in_parents,natl_pride_in_children,natl_aging_parents_live_with_children,natl_children_live_at_home_marraige,indv_pride_in_parents,indv_pride_in_children,indv_aging_parents_live_with_children,indv_children_live_at_home_marraige
0,-1.0,-1.058824,-0.555556,-2.0,-0.5,0.0,0.0,0.0,-0.333333,0.0,...,1.0,0.0,0.5,0.0,0.666667,0.5,0.5,0.0,0.666667,0.333333
1,-1.0,-0.588235,-0.500000,-1.0,-0.5,0.0,0.0,1.0,-0.666667,0.0,...,0.0,0.0,0.0,0.0,0.666667,0.5,0.5,0.0,0.666667,0.333333
2,-1.0,-0.529412,-0.277778,-2.0,-0.5,0.0,0.0,0.0,-0.333333,0.0,...,-1.0,0.0,0.0,0.0,0.333333,0.5,0.5,0.0,0.666667,0.333333
3,-1.0,-0.529412,-0.277778,-1.0,0.0,0.0,0.0,1.0,-1.333333,0.0,...,0.0,0.0,-0.5,0.0,0.666667,0.0,0.5,0.0,0.666667,0.333333
4,-1.0,-0.647059,-0.166667,-2.0,-0.5,-1.0,1.0,0.5,-1.333333,0.0,...,0.0,0.0,-0.5,-1.0,0.000000,0.0,0.5,0.0,0.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,-1.0,-0.411765,-0.277778,0.0,1.0,0.0,2.0,0.0,0.666667,0.0,...,1.0,0.0,0.0,-1.0,0.333333,-2.5,0.0,-1.0,0.333333,-1.666667
7174,-1.0,-0.176471,-0.333333,2.0,1.5,0.0,0.0,0.0,0.666667,-2.0,...,1.0,0.0,0.0,-1.0,0.333333,-0.5,-0.5,-1.0,0.666667,0.333333
7175,-1.0,-0.411765,-0.555556,-1.0,0.0,0.0,1.0,1.5,-0.333333,0.0,...,1.0,0.0,0.5,0.0,-0.333333,0.0,0.5,-1.0,-0.333333,0.000000
7176,-1.0,-0.705882,-0.500000,0.0,0.5,0.0,1.0,1.5,0.000000,0.0,...,1.0,0.0,0.5,-1.0,0.000000,-1.0,-0.5,-2.0,0.333333,-0.333333


In [5]:
vif = pd.DataFrame()
vif["feature"] = X.columns
vif["VIF"]=[variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif

# Is this done correctly? If so, do I basically keep everything? (except possibly duration)

,feature,VIF
0,sex,1.628782
1,age,4.693528
2,marriage_duration_years,5.128740
3,num_children_total,2.119964
4,num_children_inhome,1.877937
5,edu_level,1.651412
6,material_situation,1.245760
7,religiosity,1.129907
8,pension,1.204329
9,enjoy_spouse_company,2.733095


In [6]:
y.replace(to_replace=[-3,-2,-1,0,1,2], value=0, inplace=True)
y.replace(to_replace=[3], value=1, inplace=True)
y.value_counts()

0    4442
1    2736
Name: marital_satisfaction, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25)

In [11]:
log_reg = sm.Logit(y_train, X_train).fit()


Optimization terminated successfully.
         Current function value: 0.503170
         Iterations 7


In [12]:
y_pred = log_reg.predict(X_test)

In [9]:
rf=ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

0.7738161559888579


In [10]:
feature_importances = pd.DataFrame(rf.feature_importances_, index=X_train.columns, columns=['importance'])
feature_importances.sort_values(by=['importance'], ascending=False)

,importance
happiness,0.080470
age,0.074062
marriage_duration_years,0.070473
spouse_romance,0.059366
pension,0.047046
religiosity,0.045715
natl_aging_parents_live_with_children,0.043939
spouse_enjoy_doing_things_together,0.042499
spouse_attraction,0.041023
indv_aging_parents_live_with_children,0.040279


#### This has me wondering about prediction based on the perception of control. Does it make sense to predit satis based on immutable characteristic such as sex? Or one that will change, but doesn't get to be controlled, such as years married?